In [ ]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
from spellchecker import SpellChecker
from typing import List

In [ ]:
# Estableciendo directorio
tar = os.chdir(r"undisclosed")

# Leyendo base
df_contacts = pd.read_csv("undisclosed")

# Seleccionando sample
#df_contacts = df_contacts.sample(n = 2000)

In [ ]:
### Cambiando nombres a columnas

# Consiguiendo una lista con nombres originales de columnas
df_contacts_columns_original = list(df_contacts.columns.values)

# Creando una lista con nuevos nombres de columnas

df_contacts_columns_new = [
    'email',
    'nombre',
    'apellido',
    'fecha_nacimiento',
    'institucion',
    'profesion',
    'tel',
    'universidad',
    'carrera',
    'c1',
    'c2',
    'c3',
    'genero',
    'email_alternativo',
    'member_rating',
    'optin_time',
    'optin_ip',
    'confirm_time',
    'confirm_ip',
    'latitude',
    'longitude',
    'gmtoff',
    'dstoff',
    'timezone',
    'cc',
    'region',
    'last_changed',
    'leid',
    'euid',
    'notes',
    'tags'
]


# Renombrando columnas
dict_columns = dict(zip(df_contacts_columns_original, df_contacts_columns_new))
df_contacts = df_contacts.rename(columns=dict_columns)

df_contacts.head()

### Revisando NAs y data types.

In [ ]:
data = []

for column in df_contacts.columns:
    row = [
        f"{column}",
        round(1 - (df_contacts[column].isna().sum()/len(df_contacts)), ndigits = 2),
        df_contacts[column].dtype,
        type(df_contacts[column].iloc[0])
    ]
    data.append(row)

col_header = [
    'columna', 
    'porcentaje_respuesta', 
    'data_type',
    'data_type2'
]
    
tabla_descriptiva = pd.DataFrame(data, columns = col_header) 
tabla_descriptiva = tabla_descriptiva.sort_values(by = 'porcentaje_respuesta', ascending = False).reset_index(drop = True)
tabla_descriptiva

In [ ]:
### Limpiando y ordenando

In [ ]:
# Convirtiendo formato fecha nacimiento
df_contacts['fecha_nacimiento'] = pd.to_datetime(df_contacts['fecha_nacimiento'], 
                                                 format = '%d/%m/%Y', 
                                                 errors = 'coerce'
                                                )

# Eliminando hora
df_contacts['fecha_nacimiento_date'] = df_contacts.fecha_nacimiento.dt.date
# Creando año de nacimiento
df_contacts['anyo_nacimiento'] = df_contacts.fecha_nacimiento.dt.year
df_contacts['fecha_nacimiento'] = df_contacts['fecha_nacimiento_date']
df_contacts = df_contacts.drop(['fecha_nacimiento_date'], axis = 1)

In [ ]:
# Poniendo mayúsculas en nombres y apellidos
for column in ['nombre', 'apellido']:
    df_contacts[column] = df_contacts[column].str.title()

# Poniendo todos los emails en minúscula
df_contacts['email'] = df_contacts.email.str.lower()

# Cambiando columnas de texto a minúscula para facilitar modificación
for column in ['profesion', 'institucion']:
    df_contacts[column] = df_contacts[column].str.lower()

# Cambiando códigos a mayúscula
for column in ['cc', 'region']:
    df_contacts[column] = df_contacts[column].str.upper()

In [ ]:
# Limpiando teléfonos

# Remove any character that is not a '+' or digit from each string
df_contacts["tel"] = df_contacts["tel"].str.replace(r"[^\d+]", "", regex=True)

# If a cell contains 9 characters and the first character is '9', add '+56' to the beginning
df_contacts.loc[(df_contacts["tel"].str.len() == 9) & (df_contacts["tel"].str.startswith("9")), "tel"] = "+56" + df_contacts["tel"]

# If a cell contains 8 digits and no other characters, add '+569' to the beginning
df_contacts.loc[(df_contacts["tel"].str.len() == 8) & (df_contacts["tel"].str.isnumeric()), "tel"] = "+569" + df_contacts["tel"]

# If a cell contains 11 characters and the first three characters are '569', add '+' to the beginning
df_contacts.loc[(df_contacts["tel"].str.len() == 11) & (df_contacts["tel"].str.startswith("569")), "tel"] = "+" + df_contacts["tel"]

### Chequeando otra vez

In [ ]:
data = []

for column in df_contacts.columns:
    row = [
        f"{column}",
        round(1 - (df_contacts[column].isna().sum()/len(df_contacts)), ndigits = 2),
        df_contacts[column].dtype,
        type(df_contacts[column].iloc[0])
    ]
    data.append(row)

col_header = [
    'columna', 
    'porcentaje_respuesta', 
    'data_type',
    'data_type2'
]
    
tabla_descriptiva = pd.DataFrame(data, columns = col_header) 
tabla_descriptiva = tabla_descriptiva.sort_values(by = 'porcentaje_respuesta', ascending = False).reset_index(drop = True)
tabla_descriptiva

### Preprocesando columnas de texto

In [ ]:
# Sustituyendo caracteres por espacios

def replace_chars_with_spaces(df: pd.DataFrame, col_name: str) -> List[str]:
    # Check that the specified column contains string values
    if df[col_name].dtype != "object":
        raise ValueError(f"The specified column '{col_name}' does not contain string values")

    # Regex pattern that matches any non-letter, non-digit character that is not an accented character
    pattern = re.compile(r"[^a-zA-Z0-9áéíóúüÁÉÍÓÚÜñÑ ]")

    # List to store cleaned strings
    cleaned_strings = []

    # Iterate through each cell in the column
    for cell in df[col_name]:
        # Check if the cell is None or not a string
        if cell is None or not isinstance(cell, str):
            cleaned_strings.append(None)
        else:
            # Check if the cell contains only whitespace characters
            if cell.strip() == "":
                cleaned_strings.append(None)
            else:
                # Replace all non-letter, non-digit characters that are not accented characters with spaces
                cleaned_string = re.sub(pattern, lambda m: ' ' if not re.match(r'[áéíóúüÁÉÍÓÚÜñÑ]', m.group(0)) else m.group(0), cell)
                cleaned_strings.append(cleaned_string)

    # Replace the original column with the cleaned strings
    #df[col_name] = cleaned_strings

    # Return the cleaned strings as a list
    return cleaned_strings

In [ ]:
df_contacts['profesion_procesada'] = replace_chars_with_spaces(df_contacts, 'profesion')
df_contacts['institucion_procesada'] = replace_chars_with_spaces(df_contacts, 'institucion')

### Reemplazando palabras clave

In [ ]:
def replace_words(df, column_name, replacements):
    """
    Replaces all occurrences of whole words specified in the `replacements` dictionary with their corresponding
    replacement values in the specified column of the DataFrame.
    
    Parameters:
        df (pandas.DataFrame): The DataFrame containing the column to modify.
        column_name (str): The name of the column to modify.
        replacements (dict): A dictionary mapping words to their replacement values.
        
    Returns:
        pandas.DataFrame: The modified DataFrame.
    """
    # create a copy of the DataFrame to avoid modifying the original
    modified_df = df.copy()
    
    # replace all occurrences of whole words specified in the `replacements` dictionary
    for old_word, new_word in replacements.items():
        # construct a regular expression pattern to match whole words
        pattern = r'\b{}\b'.format(old_word)
        # use the pattern with the `replace()` method to replace whole words only
        modified_df[column_name] = modified_df[column_name].str.replace(pattern, new_word)
    
    return modified_df

In [ ]:
# Diccionario con palabras clave a reemplazar
palabras_profesion_reemplazo = {
    'ing': 'ingeniero',
    'vp': 'vicepresidente',
    'jubido': 'jubilado',
    'ceo': 'gerente',
    'lic': 'licenciado',
    'adm': 'administrador',
    'manager': 'gerente',
    'dr': 'doctor',
    'pensionado': 'jubilado',
    'pensionada': 'jubilado',
    'cfo': 'gerente',
    'gte': 'gerente',
    'cto': 'gerente',
    'tec': 'técnico',
    'docente': 'profesor',
    'academica': 'profesor',
    'académica': 'profesor',
    'academico': 'profesor',
    'académico': 'profesor',
    'cio': 'gerente',
    'gg': 'gerente general'
}
palabras_institucion_reemplazo = {'u': 'universidad'
                                 }

In [ ]:
df_contacts = replace_words(df_contacts, 'profesion_procesada', palabras_profesion_reemplazo)

In [ ]:
df_contacts.to_excel('df_contacts_cleaning_test1.xlsx')

### Corrigiendo errores de ortografía en columnas con texto

In [ ]:
# Crear diccionario con palabras que no deben ser corregidas

correct_spelling_list = [
    'cientista',
     'rrhh',
     'gorbea',
     'psicopedagoga',
     'tecnólogo',
     'seremi',
     '3er',
     'msc',
     'luksic',
     'free lancer',
     'arauco',
     'unep',
     'h2v',
     'policy',
     'pyme',
     'faenero',
     'demógrafa',
     'coaching',
     'gestora',
     'rrpp'
]    

known_correct_spellings = dict(zip(correct_spelling_list, correct_spelling_list))

In [ ]:
# Definiendo función correctora
def correct_spelling(df, column_name, ignore_words=None):
    # initialize spellchecker
    spell = SpellChecker(language='es')

    # initialize correction counter dictionary
    correction_counter_dict = {}
    
    # create a copy of the dataframe to avoid modifying the original
    corrected_df = df.copy()

    # iterate over each cell in the specified column and correct its spelling
    for index, row in corrected_df.iterrows():
        # get the text in the current cell
        text = row[column_name]

        # check if the cell is NaN or not of type string
        if isinstance(text, str):
            # split the text into words
            words = text.split()

            # correct the spelling of each word, ignoring certain words if specified
            corrected_words = []
            for word in words:
                if ignore_words and word in ignore_words:
                    corrected_word = word
                elif word in known_correct_spellings:
                    corrected_word = known_correct_spellings[word]
                else:
                    corrected_word = spell.correction(word)
                    if corrected_word is None:
                        corrected_word = ''
                    if word != corrected_word:
                        if corrected_word in correction_counter_dict.keys():
                            correction_counter_dict[corrected_word] += 1
                        else:
                            correction_counter_dict[corrected_word] = 1
                corrected_words.append(corrected_word)
            
            try:
                # join the corrected words back into a single string
                corrected_text = ' '.join(corrected_words)
            except:
                print(index, corrected_words)
                
            # update the cell in the dataframe with the corrected text
            corrected_df.at[index, f'{column_name}_corregida'] = corrected_text

    return corrected_df, correction_counter_dict

In [ ]:
# Aplicando corrección de ortografía
df_contacts, correction_counter = correct_spelling(df_contacts, 'profesion_procesada')

In [ ]:
# Capitalizando columnas de texto
for column in ['profesion', 'institucion', 'profesion_procesada', 'institucion_procesada', 'profesion_procesada_corregida']:
    df_contacts[column] = df_contacts[column].str.capitalize()

In [ ]:
df_contacts.columns

In [ ]:
df_contacts.to_excel('df_contacts_cleaned.xlsx')

In [ ]:
df_contacts.to_csv('df_contacts_cleaned.csv')